# Cinemática de un Movimiento Curvilíneo: Aceración normal y tangente

## Ejemplo 2

Un automóvil viaja a velocidad constante de 240 km/h en la pista descrita por la elipse en la imagen. Encuentre la aceleración experimentada por el vehículo en el punto B. Haga lo mismo para el punto A. ¿Cómo se compara la aceleración en ambos casos?  

<div style="text-align: center">
<!-- https://drive.google.com/uc?id=19xRm_SiE_ErY32WvM2NCt15mWwUImVmT -->
<img src = "https://external-content.duckduckgo.com/iu/?u=http%3A%2F%2Fdrive.google.com/uc?id=19xRm_SiE_ErY32WvM2NCt15mWwUImVmT" alt="Ejercicio 1" width=1000>
</div>


In [1]:
%%capture
# @title Latex para Google colab


# !apt-get update
# !apt-get install -y texlive-latex-base texlive-fonts-recommended texlive-fonts-extra dvipng cm-super
# !apt-get install -y texlive-latex-extra dvipng cm-super



In [2]:
# @title Drive para Google Colab

# import sys
# from google.colab import drive
# drive.mount('/content/drive')

# # Agregar la carpeta donde está helpers.py
# sys.path.append('/content/drive/MyDrive/ColabNotebooks/Unitec/Dinámica/'

In [3]:
%%capture
# @title Dependencias Python

import matplotlib.pyplot as plt
import matplotlib
import sympy as sp
import sys
import importlib
from IPython.display import Latex, clear_output
import numpy as np
from scipy.optimize import bisect
import ipywidgets as widgets

# %matplotlib inline
%matplotlib ipympl


# Agregar la carpeta donde está helpers.py
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Unitec/Dinámica/')

import helpers as hl
importlib.reload(hl)

# hl.reset_equations()

sp.init_printing()

In [4]:
%%capture
v = 240 * (1000) * (1/3600)
t, xi, theta = sp.symbols(r't \xi \theta', real=True)
s_t = sp.integrate(v, t)
s_t, v

In [5]:
v_t_lamb = hl.Lamb(v, t)

In [6]:
%%capture
a = 4000
b = 2000
e = sp.sqrt(1-((b/a)**2))
m = 1 - (a**2/b**2)
e**2, m

In [7]:
%%capture
L = 4*b*sp.elliptic_e(m)
L/v

In [8]:
%%capture
x_xi = a*sp.cos(xi)
y_xi = b*sp.sin(xi)
x_xi, y_xi

In [9]:
%%capture
Eq_theta_phi = sp.trigsimp(sp.Eq(sp.tan(theta), y_xi/x_xi))
Eq_theta_phi

In [10]:
%%capture
xi_theta = sp.solve(Eq_theta_phi, xi)[0]
xi_theta

In [11]:
%%capture
xi_theta_mon = sp.Piecewise(
  (sp.atan((a/b)*sp.tan(theta)), sp.And(theta >= -sp.pi/2, theta < sp.pi/2)),
  (sp.atan((a/b)*sp.tan(theta)) + sp.pi, sp.And(theta >= sp.pi/2, theta < 3*sp.pi/2)),
  (sp.atan((a/b)*sp.tan(theta)) + 2*sp.pi, True)
)
sp.plot(xi_theta_mon, (theta, 0, 2*sp.pi))
xi_theta_mon

In [12]:
%%capture
s_theta = b*sp.elliptic_e(xi_theta_mon, m)
s_theta

In [13]:
%%capture
Eq_st_stheta = sp.Eq(s_t, s_theta)
Eq_st_stheta

In [14]:
%%capture
s_theta_func = sp.symbols('s_theta', cls = sp.Function)
s_theta_func = s_theta_func(theta)
Eq_st_sthetafunc = sp.Eq(s_t, s_theta_func)
time_theta = sp.solve(Eq_st_sthetafunc, t)[0].subs([(s_theta_func, s_theta)])
time_theta_lamb = lambda th: float(sp.lambdify(theta, time_theta, modules=["mpmath", "numpy"])(th))
time_theta

In [15]:
def theta_t(time):
  return float(sp.nsolve(Eq_st_stheta.subs([(t, time)]), 0.5))

In [16]:
# invertir time_theta numéricamente, para obtener theta en términos del tiempo:
theta_vals = np.linspace(-(1/4)*np.pi, 2*np.pi, 5000)
t_vals = [time_theta_lamb(th) for th in theta_vals]

theta_t_numeric = lambda time: np.interp(time, t_vals, theta_vals)

In [17]:
%%capture
x_theta = x_xi.subs([(xi, xi_theta_mon)])#*sp.sign(sp.cos(theta))
y_theta = y_xi.subs([(xi, xi_theta_mon)])#*sp.sign(sp.cos(theta))

sp.plot(x_theta, (theta, 0, 2*sp.pi))
sp.plot(y_theta, (theta, 0, 2*sp.pi))
x_theta, y_theta

In [18]:
x_theta_lamb = hl.Lamb(x_theta, theta)
def x_theta_f(theta):
  return x_theta_lamb(theta)

y_theta_lamb = hl.Lamb(y_theta, theta)
def y_theta_f(theta):
  return y_theta_lamb(theta)

In [19]:
dydxi_xi = sp.diff(y_xi,xi)
dxdxi_xi = sp.diff(x_xi,xi)
dxidx_xi = 1/dxdxi_xi
dydx_xi = sp.trigsimp(dydxi_xi*dxidx_xi)#.subs([(xi, xi_theta_mon)])
dydx2_xi = sp.trigsimp(dxidx_xi*sp.diff(dydx_xi, xi))

# dydxi_xi, dxidx_xi, dydx_xi,dydx2_xi

dydx_theta = sp.trigsimp(dydx_xi.subs([(xi, xi_theta_mon)]))
dydx2_theta = sp.trigsimp(dydx2_xi.subs([(xi, xi_theta_mon)]))#*sp.sign(sp.cos(theta))

dydx_theta_lamb = hl.Lamb(dydx_theta, theta)
dydx2_theta_lamb = hl.Lamb(dydx2_theta, theta)



In [20]:
dydx_theta_lamb = hl.Lamb(dydx_theta, theta)
dydx_t = lambda time: dydx_theta_lamb(theta_t(time))

In [21]:
%%capture
sp.plot(dydx_theta, (theta, 0.5*sp.pi - 1.5, 0.5*sp.pi + 1.5));#, ylim = (-.00015, .000))

In [22]:
%%capture
eps = 0.000001
sp.plot(dydx2_theta, (theta, 0.5*sp.pi - 0.75, 0.5*sp.pi + 0.75), ylim = (-.00025, .000))
sp.limit(dydx2_theta, theta, 0.5*sp.pi, dir='+-').evalf()

In [23]:
%%capture
dydx2_theta, sp.limit(dydx2_theta, theta, .5*sp.pi, dir='+').evalf(), dydx2_theta.subs([(theta, .5*sp.pi)]).evalf()
# dydx2_theta

In [24]:
def sign_dxdt(time, eps_t = 0.001):
  t1 = time
  t2 = time + eps_t
  return np.sign(x_t(t2) - x_t(t1))

def concavity_t(time):
  return np.sign(dydx2_theta_lamb(theta_t(time)))

In [25]:
# def sign_dxdt(time, eps_t = 0.001):
#   t1 = time
#   t2 = time + eps_t
#   return np.sign(x_t(t2) - x_t(t1))

# def concavity_t(time):
#   return np.sign(dydx2_theta_lamb(theta_t(time)))


def sign_dxdt_theta(theta, eps_t = 0.001):
  time1 = time_theta_lamb(theta)
  theta1 = theta

  time2 = time1 + eps_t
  theta2 = theta_t_numeric(time2)
  return np.sign(x_theta_lamb(theta2) - x_theta_lamb(theta1))


def concavity_theta(theta):
  return np.sign(dydx2_theta_lamb(theta))
  

In [26]:
%%capture
dydxi2_xi = sp.diff(dydxi_xi, xi)
dxdxi2_xi = sp.diff(dxdxi_xi, xi)
rho_xi = sp.trigsimp((((dxdxi_xi**2) + (dydxi_xi**2))**(3/2))/
                     (sp.Abs((dydxi2_xi*dxdxi_xi) - (dxdxi2_xi*dydxi_xi))))
rho_theta = sp.trigsimp(rho_xi.subs([(xi, xi_theta_mon)]))
rho_theta

In [27]:
rho_theta_lamb = hl.Lamb(rho_theta, theta)
def rho_theta_f(theta):
  return rho_theta_lamb(theta)

In [28]:
a_t = sp.diff(v, t)
a_t_t_lamb = hl.Lamb(a_t, t)

In [29]:
v_theta_lamb = hl.Lamb(v, theta)
a_t_theta_lamb = hl.Lamb(a_t, theta)

In [30]:
def u_vecs_theta(theta):
  dy = dydx_theta_lamb(theta)
  sign_dx = sign_dxdt_theta(theta)
  conc = concavity_theta(theta)
  u_t = sign_dx*(1/np.sqrt(1+(dy**2)))*np.array([1, dy])
  u_n = (sign_dx*conc)*np.array([-u_t[1], u_t[0]])
  return u_t, u_n
  
def u_t_theta(theta):
  dy = dydx_theta_lamb(theta)
  return sign_dxdt_theta(theta)*(1/np.sqrt(1+(dy**2)))*np.array([1, dy])

def u_n_theta(theta):
  ut = u_t_theta(theta)
  return (sign_dxdt_theta(theta)*concavity_theta(theta))*np.array([-ut[1], ut[0]])



def v_theta(theta):
  return v_theta_lamb(theta)

def a_t_theta(theta):
  return a_t_theta_lamb(theta)

def a_n_theta(theta):
  rho = rho_theta_f(theta)
  return v_theta(theta)**2/rho, rho


def rvec_t(theta):
  xt = x_theta_f(theta)
  yt = y_theta_f(theta)
  return np.array([xt, yt])

def vvec_t(theta):
  return v_theta(theta)*u_vecs_theta(theta)[0]

def avec_t(theta):
  ut, un = u_vecs_theta(theta)
  at = a_t_theta(theta)
  an, rho = a_n_theta(theta)
  return at*ut + an*un, at*ut, an*un, un, rho

# def at_vec_t(theta):
#   return a_t_theta(theta)*u_t_theta(theta)

# def an_vec_t(theta):
#   return a_n_theta(theta)*u_n_theta(theta)

In [142]:
# @title Elegir tema para gráficas

''' Elegir entre:
tema = 'blanco'
tema = 'cobalto'
tema = 'catpuccin latte'
'''

tema = 'catpuccin latte'

In [143]:
hl.aplicar_tema(tema)

In [144]:
%%capture
# @title Parámetros de gráfica

ax_bound = [-4100, 4100, -2100, 2100]
ax_ratio = 1 # graphic ratio of y to x units
ax_zoom_bound = [0.5, 0.1, .4, .4]
scale_zoom = 3
scale_v = 10
scale_a = 500
zoom_scale_v = 0.5
zoom_scale_a = 0.5

ver_pad = 1.25
hor_pad = 1.
ax_aspect = ((ax_bound[3] - ax_bound[2])*ver_pad)/((ax_bound[1] - ax_bound[0])*hor_pad)
fig_width = 12 # inches
fig_height = fig_width * ax_aspect
dpi = 100


# Crear gráfica
fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_height), dpi = dpi)
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False


ax.set_aspect("equal", adjustable='datalim')

ax.set_xlabel(r'$x\, \mathrm{[\,m\,]}$')
ax.set_ylabel(r'$y\, \mathrm{[\,m\,]}$')
ax.axhline(y = 0, color = plt.rcParams['grid.color'], linestyle = '-', lw = 1.5)
ax.axvline(x = 0, color = plt.rcParams['grid.color'], linestyle = '-', lw = 1.5)


ax.set_aspect(ax_ratio, adjustable='box')
ax.axis(ax_bound)
ax.grid(True)

legend_params = dict(loc = 'lower left', bbox_to_anchor = (.0, 1.0, 1.0, 0.1), ncol = 5)

In [145]:
%%capture
# @title Gráficas Interactivas

t0 = 5

theta_range = np.arange(0, 2*np.pi, 0.01)


x_path = x_theta_lamb(theta_range)
y_path = y_theta_lamb(theta_range)




particle_path = ax.plot(x_path, y_path, 'C0',lw = 2)




r_0 = rvec_t(t0)
r_label = r'$\displaystyle \boldsymbol{{\vec{{r}}}} = \left[\begin{{array}} {{c}} {:.2f} \, \mathrm{{m}} \\ {:.2f} \, \mathrm{{m}} \end{{array}}\right] = {:.2f} \, \mathrm{{m}}$'.format(*r_0, np.linalg.norm(r_0))
rV = hl.FancyVector2D(ax, 0, 0, *r_0, color='C1', label=r_label)



v_0 = vvec_t(t0)
v_label = r'$\displaystyle \boldsymbol{{\vec{{v}}}} = \left[\begin{{array}} {{c}} {:.2f} \, \mathrm{{m/s}} \\ {:.2f} \, \mathrm{{m/s}} \end{{array}}\right] = {:.2f} \, \mathrm{{m/s}}$'.format(*v_0, np.linalg.norm(v_0))
vV = hl.FancyVector2D(ax, x1 = v_0[0]*scale_v, y1 = v_0[1]*scale_v, color='C2', previous = rV, label=v_label)




a_0, at_0, an_0, un_0, rho_0 = avec_t(t0)
# at_0 = at_vec_t(t0)
# an_0 = an_vec_t(t0)

a_label = r'$\displaystyle ||\boldsymbol{{\vec{{a}}}}|| = {:.2f} \, \mathrm{{m/s^2}} $'.format(np.linalg.norm(a_0))
a_t_label = r'$\displaystyle \boldsymbol{{a_t}} = {:.2f} \, \mathrm{{m/s^2}} $'.format(np.linalg.norm(at_0))
a_n_label = r'$\displaystyle \boldsymbol{{a_n}} = {:.2f} \, \mathrm{{m/s^2}} $'.format(np.linalg.norm(an_0))

a_tV = hl.FancyVector2D(ax, x1 = at_0[0]*scale_a, y1 = at_0[1]*scale_a, color='C4', linewidth = 1.5, head_length = 15, head_width = 10, head_tail = 3, previous = rV, label= a_t_label)
a_nV = hl.FancyVector2D(ax, x1 = an_0[0]*scale_a, y1 = an_0[1]*scale_a, color='C5', linewidth = 1.5, head_length = 15, head_width = 10, head_tail = 3, previous = a_tV, label = a_n_label)
aV = hl.FancyVector2D(ax, x1 = a_0[0]*scale_a, y1 = a_0[1]*scale_a, color='C3', previous = rV, label= a_label)

osc_circle = hl.OsculatingCircle2D(ax, *r_0, un_0, rho_0)

leg = ax.legend(**legend_params)
leg_handles, leg_labels = ax.get_legend_handles_labels()





In [146]:
%%capture
# @title redibujar vectores
# %matplotlib ipympl

def update_Vectors(r, v, a, a_t, a_n, circle):
  def update(t = t0):
    theta = theta_t_numeric(t)

    rt = rvec_t(theta)
    vt = vvec_t(theta)
    at, at_t, at_n, un, rho = avec_t(theta)
    # at_t = at_vec_t(theta)
    # at_n = an_vec_t(theta)


    r.set_end(*rt)
    circle.update(*rt, un, rho)

    v.set_XY(*vt*scale_v)
    a.set_XY(*at*scale_a)
    a_t.set_XY(*at_t*scale_a)
    a_n.set_XY(*at_n*scale_a)


    r_label = r'$\displaystyle \boldsymbol{{\vec{{r}}}} = \left[\begin{{array}} {{c}} {:.2f} \, \mathrm{{m}} \\ {:.2f} \, \mathrm{{m}} \end{{array}}\right] = {:.2f} \, \mathrm{{m}}$'.format(*rt, np.linalg.norm(rt))
    v_label = r'$\displaystyle \boldsymbol{{\vec{{v}}}} = \left[\begin{{array}} {{c}} {:.2f} \, \mathrm{{m/s}} \\ {:.2f} \, \mathrm{{m/s}} \end{{array}}\right] = {:.2f} \, \mathrm{{m/s}}$'.format(*vt, np.linalg.norm(vt))
    a_t_label = r'$\displaystyle \boldsymbol{{a_t}} = {:.2f} \, \mathrm{{m/s^2}} $'.format(np.linalg.norm(at_t))
    a_n_label = r'$\displaystyle \boldsymbol{{a_n}} = {:.2f} \, \mathrm{{m/s^2}} $'.format(np.linalg.norm(at_n))
    a_label = r'$\displaystyle ||\boldsymbol{{\vec{{a}}}}|| = {:.2f} \, \mathrm{{m/s^2}} $'.format(np.linalg.norm(at))

    labels = [
      r_label,
      v_label,
      a_t_label,
      a_n_label,
      a_label,
    ]

    vectors = [
      r,
      v,
      a_t,
      a_n,
      a
    ]

    visible_labels = [l for l, vec in zip(labels, vectors) if vec.main]

    # La leyenda inicial tiene demasiados elementos, entonces hay que limitar
    # el número de textos en el primer enumerate
    N = len(visible_labels)


    for i, text in enumerate([*ax.get_legend().get_texts()[0:N]]):
      text.set_text(visible_labels[i])


    # Dibujar
    clear_output(wait = True)
    fig.canvas.draw_idle()
  return update

In [147]:
%%capture
# @title elegir qué es visible
def update_Visibility(r, v, a, a_t, a_n, circle, time):
  def update_V(visible):

    visible_legends = [
        visible == 'r(t)',
        visible == 'v(t)',
        visible == 'a(t)',
        visible == 'a(t)',
        visible == 'a(t)',
    ]

    if visible == 'r(t)':
      # Set Colors
      r.set_color('C1')

      # Set Visibles
      r.set_visible(True)
      v.set_visible(False)
      a_t.set_visible(False)
      a_n.set_visible(False)
      a.set_visible(False)
      circle.set_visible(True)

      # Set Mains
      r.set_main(True)
      v.set_main(False)
      a_t.set_main(False)
      a_n.set_main(False)
      a.set_main(False)
    elif visible == 'v(t)':
      # Set Colors
      r.set_color(hl.toward_gray('C1', 0.5, ax.get_facecolor()))
      v.set_color('C2')

      # Set Visibles
      r.set_visible(True)
      v.set_visible(True)
      a_t.set_visible(False)
      a_n.set_visible(False)
      a.set_visible(False)
      circle.set_visible(False)

      # Set Mains
      r.set_main(False)
      v.set_main(True)
      a_t.set_main(False)
      a_n.set_main(False)
      a.set_main(False)
    else:
      # Set Colors
      r.set_color(hl.toward_gray('C1', 0.75, ax.get_facecolor()))
      v.set_color(hl.toward_gray('C2', 0.7, ax.get_facecolor()))

      # Set Visibles
      r.set_visible(True)
      v.set_visible(True)
      a_t.set_visible(True)
      a_n.set_visible(True)
      a.set_visible(True)
      circle.set_visible(False)

      # Set Mains
      r.set_main(False)
      v.set_main(False)
      a_t.set_main(True)
      a_n.set_main(True)
      a.set_main(True)


    handles, labels = ax.get_legend_handles_labels()
    handles_vis = [h for h, vis in zip(handles, visible_legends) if vis]
    labels_vis = [l for l, vis in zip(labels, visible_legends) if vis]
    ax.legend(handles_vis, labels_vis, **legend_params)

    # with out:
    #   out.clear_output(wait = True)
    update_Vectors(r, v, a, a_t, a_n, circle)(time)

  return update_V

In [148]:
import solara
from io import BytesIO
solara.lab.theme.dark_effective = True
t_0 = 0.0001

t_reactive = solara.reactive(t_0)
use_solara_image = solara.reactive(False)
graph_types = ['r(t)', 'v(t)', 'a(t)']
graph_type_reactive = solara.reactive(graph_types[0])



update_my_Vectors = update_Vectors(rV, vV, aV, a_tV, a_nV, osc_circle)


update_my_Vectors(t_0)
update_Visibility(rV, vV, aV, a_tV, a_nV, osc_circle, t_reactive.value)(graph_types[0])

@solara.component
def Page():
  update_Visible = update_Visibility(rV, vV, aV, a_tV, a_nV, osc_circle, t_reactive.value)
  # t_reactive, set_t = solara.use_state(0.0)
  def on_change_time(new_t):
    update_my_Vectors(new_t)

  def on_change_visible(new_v):
    update_Visible(new_v)

  def fig_to_bytes(time):
    on_change_time(time)
    buf = BytesIO()
    fig.savefig(buf, format="png", bbox_inches="tight")
    buf.seek(0)
    return buf.read()

  with solara.Column(margin= 0, gap= "0px", style={"background": matplotlib.rcParams['figure.facecolor'],
                                                  "width": "100%",
                                                  # "height": "100%",
                                                  "margin": "0",
                                                  "padding": "1rem",
                                                  "display": "flex",
                                                  "flexDirection": "column",
                                                  "boxSizing": "border-box",
                                                  }) as App:
    solara.Checkbox(label= 'Reescalar figura para pantallas pequeñas', value= use_solara_image,
                              on_value= lambda v: use_solara_image.set(v))
    with solara.Column(margin= 4, style={"background": matplotlib.rcParams['figure.facecolor']}):
      solara.SliderFloat("t", value= t_reactive, min= t_0, 
                                           max= 295., step= 5, 
                                           on_value= lambda t: t_reactive.set(t),
                                           thumb_label= False, tick_labels= False)
    with solara.ToggleButtonsSingle(value= graph_type_reactive, on_value= on_change_visible, style = {"width": "300px"}):
      for i in range(len(graph_types)):
        solara.Button(graph_types[i], style = {"width": "100px"})
    
    # solara.FigureMatplotlib(fig)
    # solara.Image(fig_to_bytes())
    # solara.HTML(ipycanvas_widget)
    # return t_slider_solara
    # plt.show()
    if use_solara_image.value:
      solara.Image(fig_to_bytes(t_reactive.value))
    else:
      on_change_time(t_reactive.value)
      plt.show()
  return App
# Page()

In [149]:
Page()

Cannot show ipywidgets in text